# FISH integrated analyses

<img src= /home/luisub/Desktop/FISH_Processing/docs/code_architecture.png alt="drawing" width="1000"/>

<img src= /Users/luisaguilera/Desktop/FISH_Processing/docs/code_architecture.png alt="drawing" width="1000"/>

## Libraries

In [ ]:
# Importing libraries
import sys
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import pathlib
import warnings
warnings.filterwarnings("ignore")

## Directories Path

In [ ]:
# Deffining directories
current_dir = pathlib.Path().absolute()
fa_dir = current_dir.parents[0].joinpath('src')

# Importing fish_analyses module
sys.path.append(str(fa_dir))
import fish_analyses as fa

In [ ]:
#data_dir = current_dir.parents[0].joinpath('dataBases').joinpath('ms2_cy5_20210518')
data_dir = current_dir.parents[0].joinpath('dataBases').joinpath('GAPDH_Exon_Cy5')     # path to a folder with images.
channels_with_cytosol = [1,2]            # list or int indicating the channels where the cytosol is detectable
channels_with_nucleus = 0                # list or int indicating the channels where the nucleus is detectable
channels_with_FISH = [1,2]               # list or int with the channels with FISH spots that are used for the quantification
diamter_nucleus = 200                    # approximate nucleus size in pixels
diameter_cytosol = 250 #250              # approximate cytosol size in pixels
# Parameters for FISH detection
voxel_size_z = 500                       # Microscope conversion px to nanometers in the z axis.
voxel_size_yx = 103                      # Microscope conversion px to nanometers in the xy axis.
psf_z = 600      #350                    # Theoretical size of the PSF emitted by a [rna] spot in the z plan, in nanometers.
psf_yx = 150     #150                    # Theoretical size of the PSF emitted by a [rna] spot in the yx plan, in nanometers.
# Cluster Detection
minimum_spots_cluster = 7                # The number of spots in a neighborhood for a point to be considered as a core point (from which a cluster is expanded). This includes the point itself.
show_plots=True                          # Flag to display plots

In [ ]:
dataframe_FISH = fa.PipelineFISH(data_dir, channels_with_cytosol, channels_with_nucleus, channels_with_FISH,diamter_nucleus, diameter_cytosol, voxel_size_z, voxel_size_yx, psf_z, psf_yx, minimum_spots_cluster,show_plot=show_plots).run()

In [ ]:
dataframe_FISH.tail()

In [ ]:
# Number of cells
spot_type_selected = 0
number_cells = dataframe_FISH['cell_id'].nunique()
print(number_cells)
# Number of spots
number_of_spots_per_cell = [len( dataframe_FISH.loc[  (dataframe_FISH['cell_id']==i) &  (dataframe_FISH['is_cluster']==False) & (dataframe_FISH['is_nuc']==True) & (dataframe_FISH['spot_type']==spot_type_selected) ].spot_id) for i in range(0, number_cells)]
# Number of TS per cell
number_of_TS_per_cell = [len( dataframe_FISH.loc[  (dataframe_FISH['cell_id']==i) &  (dataframe_FISH['is_cluster']==True) & (dataframe_FISH['is_nuc']==True) & (dataframe_FISH['spot_type']==spot_type_selected) ].spot_id) for i in range(0, number_cells)]
# Number of RNA in a TS
ts_size =  dataframe_FISH.loc[   (dataframe_FISH['is_cluster']==True) & (dataframe_FISH['is_nuc']==True)  & (dataframe_FISH['spot_type']==spot_type_selected)   ].cluster_size.values
# Size of each cell
cell_size = dataframe_FISH.loc[  (dataframe_FISH['spot_id']==0)  ].cell_area_px.values

In [ ]:
def plot_probability_distribution(data_to_plot, numBins = 10, title='', xlab='', ylab='', color='r', subplots=False, show_grid=True, fig=plt.figure() ):
  #def normalization_hist (data,numBins=10):
  #  results, edges = np.histogram(data,bins=numBins, normed=False)
  #  binWidth = edges[1] - edges[0]
  #  return edges, results, binWidth
  #edges, results, binWidth = normalization_hist (data_to_plot,numBins)
  #plt.bar(edges[:-1], results*binWidth, binWidth,color=color)
  plt.hist(data_to_plot,bins=numBins,density=False,color=color)
  plt.xlabel(xlab, size=16)
  plt.ylabel(ylab, size=16)
  plt.grid(show_grid)
  plt.title(title, size=16)
  return (f)

In [ ]:
#Plotting
fig_size = (25, 5)
f = plt.figure(figsize=fig_size)
#ylab='Probability'
ylab='Frequency Count'  
# adding subplots
f.add_subplot(1,4,1) 
plot_probability_distribution( number_of_spots_per_cell, numBins=20,  title='Num Spots per cell', xlab='Number', ylab=ylab, fig=f, color='g')
f.add_subplot(1,4,2) 
plot_probability_distribution(number_of_TS_per_cell,   numBins=20,  title='num TS per cell', xlab='TS per cell', ylab=ylab, fig=f, color='g')
f.add_subplot(1,4,3) 
plot_probability_distribution(ts_size, numBins=20,    title='TS size', xlab='RNA per TS', ylab=ylab, fig=f, color='g')
f.add_subplot(1,4,4) 
plot_probability_distribution(cell_size ,  numBins=20, title='Cell size', xlab='[Pixels]', ylab=ylab, fig=f, color='g')
# plt.savefig('./results/sim_intensity_dist_ch0.png')
plt.show()